In [1]:
import pandas as pd

### Filter citation only for app_id 15...

Below cell need memory more than 15GB.

In [2]:
citations = pd.read_csv("../data/citations.csv")

In [3]:
citations.head()

,app_id,citation_pat_pgpub_id,parsed,ifw_number,action_type,action_subtype,form892,form1449,citation_in_oa
0,12000001,20060218340,20060218340,NaN,NaN,NaN,0,1,0
1,12000001,2343564,2343564,NaN,NaN,NaN,1,0,0
2,12000001,6622200,6622200,NaN,NaN,NaN,1,0,0
3,12000001,6182004,6182004,NaN,NaN,NaN,0,1,0
4,12000001,2323432,2323432,NaN,NaN,NaN,1,0,0


In [14]:
len(citations)

58862278

https://patentlibrarian.com/reference-library/u-s-application-series-codes/

Collect citation including 2017, which is, app_id begins with 15.

- 14 2013, August - 2016, Jan?
- 15 2016, Jan - Present?



In [11]:
citations_15 = citations[(citations.app_id >= 15000000) & (citations.app_id < 16000000)]

In [12]:
citations_15.head()

,app_id,citation_pat_pgpub_id,parsed,ifw_number,action_type,action_subtype,form892,form1449,citation_in_oa
57732192,15000008,8291627,8291627,NaN,NaN,NaN,1,0,0
57732193,15000008,2093620,2093620,NaN,NaN,NaN,1,0,0
57732194,15000008,9221406,9221406,NaN,NaN,NaN,1,0,0
57732195,15000008,8544198,8544198,NaN,NaN,NaN,1,0,0
57732196,15000008,7225569,7225569,NaN,NaN,NaN,1,0,0


In [13]:
len(citations_15)

1130086

In [15]:
citations_15.to_pickle("../data/citations_15.dat")

### Filer office_actions of app_id starts with 15... only

In [4]:
office_actions = pd.read_csv("../data/office_actions.csv")

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
len(office_actions)

4384532

In [6]:
office_actions.head()

,app_id,ifw_number,document_cd,mail_dt,art_unit,uspc_class,uspc_subclass,header_missing,fp_missing,rejection_fp_mismatch,...,rejection_103,rejection_112,rejection_dp,objection,allowed_claims,cite102_gt1,cite103_gt3,cite103_eq1,cite103_max,signature_type
0,14150981,100867762,CTFR,2015-10-15,2632,375,219000,0,0,0,...,1,0,0,0,0,0,0,1,2,1
1,14198961,100867788,CTFR,2015-10-15,2699,345,173000,0,0,0,...,1,0,0,0,0,0,0,0,2,1
2,13796589,100867794,CTNF,2015-10-15,3776,606,159000,0,0,0,...,1,0,0,0,0,0,0,0,3,3
3,14673475,100867844,CTNF,2015-10-15,2627,345,175000,0,0,0,...,1,1,1,0,0,0,1,0,4,1
4,14669113,100867850,CTNF,2015-10-15,2666,382,118000,0,0,0,...,0,1,1,0,0,0,0,0,0,1


In [13]:
office_15 = office_actions[(office_actions.app_id >= 15000000) & (office_actions.app_id < 16000000)]

In [14]:
len(office_15)

68010

In [16]:
office_15.to_pickle("../data/office_15.dat")

### Load citations and office actions which app_id starts with 15 only

To reduce footprint, restart kernel here and reload it.

In [50]:
citations_15 = pd.read_pickle("../data/citations_15.dat")

In [12]:
office_15 = pd.read_pickle("../data/office_15.dat")

In [4]:
office_15.head()

,app_id,ifw_number,document_cd,mail_dt,art_unit,uspc_class,uspc_subclass,header_missing,fp_missing,rejection_fp_mismatch,...,rejection_103,rejection_112,rejection_dp,objection,allowed_claims,cite102_gt1,cite103_gt3,cite103_eq1,cite103_max,signature_type
3549653,15001440,IKIANOLHRXEAPX4,CTNF,2016-02-16,3762,607,042000,0,0,0,...,0,1,0,0,0,0,0,0,0,1
3560595,15009822,IKSBCEFQRXEAPX1,CTNF,2016-02-22,2139,711,162000,0,0,0,...,1,0,0,0,0,0,0,0,2,1
3561801,15005636,IKSG9XK1RXEAPX1,CTNF,2016-02-22,3766,600,509000,0,0,0,...,0,1,1,0,0,0,0,0,0,0
3562763,15002146,IKSLCJQLRXEAPX4,CTNF,2016-02-22,3723,015,104930,0,0,0,...,0,0,1,1,0,0,0,0,0,1
3564336,15001553,IKTT4P0TRXEAPX1,CTNF,2016-02-19,1625,514,279000,0,1,0,...,0,1,0,0,0,0,0,0,0,1


### Create poor man's index files by unix command

In [2]:
import os, os.path

In [3]:
INDEX_PATH="../data/index/"

In [4]:
if not os.path.exists(INDEX_PATH):
    os.mkdir(INDEX_PATH)

In [5]:
def create_one_index(xmlrel):
    xmlpath = "../data/"+xmlrel
    indexfile=INDEX_PATH+xmlrel+".idx"
    !mkdir -p {os.path.dirname(indexfile)}
    !grep -n "doc-number" {xmlpath} | sed -e 's/\(.*\)<doc-number>\([^<]*\)<\/doc-number>.*/\1\2/' > {indexfile}

## Data setup assumption

Download application of 2017 and grants of 2012, unzip xml, and place under data/

From here: https://bulkdata.uspto.gov/#pats

### Grants 2012

Use "Patent Grant Full Text Data (No Images) (JAN 1976 - PRESENT)".

- Download all zip of https://bulkdata.uspto.gov/data/patent/grant/redbook/fulltext/2012/
   - Use shell script to download zip (https://colab.research.google.com/drive/1gJO0VQ72Xfr_SY2dwWyiTvczIv0_gd4L)
       - save to download.sh and run
- place xml under data/grants2012/
   - ex.data/grants2012/ipg120103.xml data/grants2012/ipg120221.xml
   - Total about 24GB

### Application 2017

Use "Patent Application Full Text Data (No Images) (MAR 15, 2001 - PRESENT)"

- Download all zip of https://bulkdata.uspto.gov/data/patent/application/redbook/fulltext/2017/
   - Use above colab shell script again
- place xml under data/application/2017
    - ex. data/application2017/ipa170105.xml data/appli cation2017/ipa170223.xml
    - Total about 35GB

In [6]:
import glob

In [67]:
xmls = glob.glob("../data/grants2012/*.xml")

In [68]:
xmls[0]

'../data/grants2012/ipg120911.xml'

In [69]:
xmls[0][len("../data/"):]

'grants2012/ipg120911.xml'

In [70]:
xmlrels = [xml[len("../data/"):] for xml in xmls]

In [72]:
xmlrels[0]

'grants2012/ipg120911.xml'

In [73]:
%%time

for xmlrel in xmlrels:
    create_one_index(xmlrel)

CPU times: user 4.49 s, sys: 812 ms, total: 5.3 s
Wall time: 3min 48s


In [9]:
xmls = glob.glob("../data/application2017/*.xml")
xmlrels = [xml[len("../data/"):] for xml in xmls]

In [10]:
%%time

for xmlrel in xmlrels:
    create_one_index(xmlrel)

CPU times: user 9.32 s, sys: 1.43 s, total: 10.8 s
Wall time: 7min 2s


### Load index from files

In [65]:
import glob

In [7]:
# dict format:
# key is doc-number
# val is list of [relxml, linenumber]

def build_one_index_file_to_dict(indexfile, dic):
    with open(indexfile, "r") as f:
        tupples = [l.rstrip("\n").split(":", 1) for l in f]
    relxml = indexfile[len("../data/index/"):].rstrip(".idx")
    for tup in tupples:
        dic.setdefault(tup[1], []).append([relxml, int(tup[0])])

In [16]:
def build_all_index_dict(indexfiles, dic):
    for f in indexfiles:
        build_one_index_file_to_dict(f, dic)

In [13]:
indexfiles2017 = glob.glob("../data/index/application2017/*.idx")

In [17]:
index2017 = {}

In [18]:
build_all_index_dict(indexfiles2017, index2017)

In [45]:
indexfilesGrants2012 = glob.glob("../data/index/grants2012/*.idx")

In [47]:
indexGrants2012 = {}

In [48]:
build_all_index_dict(indexfilesGrants2012, indexGrants2012)

### Filter office action only of application 2017 doc-number.

In [37]:
# key of index2017 is string because doc-number sometime contains non-int value.
# so we create string row in office action data frame to use isin method.

office_15["app_id_str"] = office_15.app_id.map(str)

In [36]:
sum(office_15.app_id_str.isin(index2017) == True)

21820

In [38]:
office2017 = office_15[office_15.app_id_str.isin(index2017)]

In [40]:
len(office2017)

21820

In [39]:
office2017.head()

,app_id,ifw_number,document_cd,mail_dt,art_unit,uspc_class,uspc_subclass,header_missing,fp_missing,rejection_fp_mismatch,...,rejection_112,rejection_dp,objection,allowed_claims,cite102_gt1,cite103_gt3,cite103_eq1,cite103_max,signature_type,app_id_str
3561801,15005636,IKSG9XK1RXEAPX1,CTNF,2016-02-22,3766,600,509000,0,0,0,...,1,1,0,0,0,0,0,0,0,15005636
3564336,15001553,IKTT4P0TRXEAPX1,CTNF,2016-02-19,1625,514,279000,0,1,0,...,1,0,0,0,0,0,0,0,1,15001553
3568839,15009367,IKZGL7O1RXEAPX4,CTNF,2016-02-25,2137,711,103000,0,0,0,...,0,1,0,0,0,0,0,0,1,15009367
3569392,15041416,IKZI6SS3RXEAPX1,CTNF,2016-02-25,2618,345,619000,0,0,0,...,0,0,0,0,0,1,0,5,1,15041416
3574015,15014088,IL0U1CK0RXEAPX5,CTNF,2016-02-26,2852,399,085000,0,0,0,...,0,0,1,0,0,0,0,2,1,15014088


In [41]:
ifw_from_OA = set(office2017.ifw_number)

In [42]:
len(list(ifw_from_OA))

21820

In [51]:
citations_15.head()

,app_id,citation_pat_pgpub_id,parsed,ifw_number,action_type,action_subtype,form892,form1449,citation_in_oa
57732192,15000008,8291627,8291627,NaN,NaN,NaN,1,0,0
57732193,15000008,2093620,2093620,NaN,NaN,NaN,1,0,0
57732194,15000008,9221406,9221406,NaN,NaN,NaN,1,0,0
57732195,15000008,8544198,8544198,NaN,NaN,NaN,1,0,0
57732196,15000008,7225569,7225569,NaN,NaN,NaN,1,0,0


In [55]:
citations_2017 = citations_15[citations_15.ifw_number.isin(ifw_from_OA)]

In [56]:
len(citations_2017)

43416

In [57]:
citations_2017.head()

,app_id,citation_pat_pgpub_id,parsed,ifw_number,action_type,action_subtype,form892,form1449,citation_in_oa
57732253,15000027,8553488,8553488,IRPCOHEPRXEAPX4,102.0,a,1,0,1
57732273,15000034,Dando US 2015/0364400,20150364400,IYD99O6CRXEAPX0,103.0,NaN,0,0,1
57732274,15000034,Sun US 2012/0181696,20120181696,IYD99O6CRXEAPX0,103.0,NaN,0,0,1
57732275,15000034,Sun US 2012/0279760,20120279760,IYD99O6CRXEAPX0,103.0,NaN,0,0,1
57732324,15000048,20140091294,20140091294,IR59S77KRXEAPX5,103.0,NaN,1,0,1


In [58]:
citations_we_have = citations_2017[citations_2017.citation_pat_pgpub_id.isin(indexGrants2012)]

In [59]:
len(citations_we_have)

4362

In [60]:
citations_we_have.head()

,app_id,citation_pat_pgpub_id,parsed,ifw_number,action_type,action_subtype,form892,form1449,citation_in_oa
57733581,15000285,4659845,4659845,IV82WOB1RXEAPX5,103.0,NaN,1,0,1
57733622,15000297,7268118,7268118,IO1DLQ1SRXEAPX3,102.0,b,0,1,1
57733623,15000297,7268118,7268118,IO1DLQ1SRXEAPX3,103.0,a,0,1,1
57733624,15000297,5578570,5578570,IO1DLQ1SRXEAPX3,103.0,a,1,0,1
57733628,15000297,5342620,5342620,IO1DLQ1SRXEAPX3,103.0,a,1,1,1


In [61]:
citations_we_have.to_pickle("../data/citations_2017_2012.dat")

In [65]:
index2017_we_need = {str(key):index2017[str(key)] for key in set(citations_we_have.app_id)}

In [66]:
len(list(index2017_we_need.keys()))

2322

In [67]:
indexGrants2012_we_need = {str(key):indexGrants2012[str(key)] for key in set(citations_we_have.citation_pat_pgpub_id)}

In [69]:
list(indexGrants2012_we_need.keys())[0:5]

['7945484', 'WO 00/54659', '20100306879', '4242108', '4619580']

In [70]:
len(list(indexGrants2012_we_need.keys()))

3071

In [71]:
indexGrants2012_we_need['7945484']

[['grants2012/ipg120417.xml', 6067926],
 ['grants2012/ipg121204.xml', 6903406],
 ['grants2012/ipg121204.xml', 6903688]]

In [73]:
## cut subtext of (target-bound):(target+bound) from very large text file
## but if (target-bound) is smaller than 1, treat as 1.
## retuned value is tupple which value is
## first cell: offset for target
## second cell: array of text, each element stand for one line.

def retrieve_subtext_fast(target, bound, fpath):
    start = max(1, target-bound)
    end = target+bound
    sed_arg = "'{},{}p'".format(start, end)
    subtext = !sed -n {sed_arg} {fpath}
    return (target - start, subtext)

In [95]:
#<us-patent-application ...>...</us-patent-application>
#<us-patent-grant ...>....</us-patent-grant>

def find_nearest_root_element(target_offset, subtext_arr, tagname):
    begin_tag = '<{}'.format(tagname)
    last_cand = -1
    for i, line in enumerate(subtext_arr):
        if i > target_offset:
            if last_cand == -1:
                raise Exception('no target tagname found around target_offset')
            return last_cand
        if begin_tag in line:
            last_cand = i
    raise Exception('no target tagname found around target_offset')        

In [101]:
def retrieve_begin_end(begin_offset, subtext_arr, tagname):
    end_tag = '</{}'.format(tagname)
    res = []
    for line in subtext_arr[begin_offset:]:
        res.append(line)
        if end_tag in line:
            return res
    raise Exception('no end tagname found')        

In [106]:
import re

In [126]:
DOC_NUMBER_PAT=re.compile(r'<doc-number>([^<]+)</doc-number>')

def find_first_doc_number(text_arr):
    for text in text_arr:
        matchObj = DOC_NUMBER_PAT.search(text)
        if matchObj:
            return matchObj.group(1)
    raise Exception("No doc-number found.")
        
    

In [127]:
find_first_doc_number(res_pat)

'20170351915'

In [131]:
# tagname is "us-patent-application" or "us-patent-grant"
#<us-patent-application ...>...</us-patent-application>
#<us-patent-grant ...>....</us-patent-grant>

def get_patent(tagname, file_path, target_offset):
    BOUND=5000
    target_mod_offset, subtext_arr = retrieve_subtext_fast(target_offset, BOUND, file_path)
    begin_pos = find_nearest_root_element(target_mod_offset, subtext_arr, tagname)
    result_patent = retrieve_begin_end(begin_pos, subtext_arr, tagname)
    docid = find_first_doc_number(result_patent)
    return docid, result_patent

In [71]:
indexGrants2012_we_need['7945484']

[['grants2012/ipg120417.xml', 6067926],
 ['grants2012/ipg121204.xml', 6903406],
 ['grants2012/ipg121204.xml', 6903688]]

In [140]:
real_id1, pat_text1 = get_patent("us-patent-grant", '../data/' + 'grants2012/ipg120417.xml', 6067926)
real_id1

'08160929'

In [141]:
real_id2, pat_text2 = get_patent("us-patent-grant", '../data/' + 'grants2012/ipg121204.xml', 6903406)
real_id2

'08326698'

In [142]:
real_id3, pat_text3 = get_patent("us-patent-grant", '../data/' + 'grants2012/ipg121204.xml', 6903688)
real_id3

'08326698'

In [146]:
with open('tmp_out.txt', 'w') as f: 
    print("\n".join(pat_text1), file=f)

In [ ]:
['grants2012/ipg120417.xml', 6067926],
 ['grants2012/ipg121204.xml', 6903406],
 ['grants2012/ipg121204.xml', 6903688]

In [129]:
a, b = res_tup

In [86]:
res_tup[1][res_tup[0]]

'<doc-number>15146848</doc-number>'

In [88]:
res_tup[1][950:1050]

['<subclass>F</subclass>',
 '<main-group>17</main-group>',
 '<subgroup>30371</subgroup>',
 '<symbol-position>L</symbol-position>',
 '<classification-value>I</classification-value>',
 '<action-date><date>20171207</date></action-date>',
 '<generating-office><country>US</country></generating-office>',
 '<classification-status>B</classification-status>',
 '<classification-data-source>H</classification-data-source>',
 '<scheme-origination-code>C</scheme-origination-code>',
 '</classification-cpc>',
 '<classification-cpc>',
 '<cpc-version-indicator><date>20130101</date></cpc-version-indicator>',
 '<section>G</section>',
 '<class>06</class>',
 '<subclass>F</subclass>',
 '<main-group>17</main-group>',
 '<subgroup>3028</subgroup>',
 '<symbol-position>L</symbol-position>',
 '<classification-value>I</classification-value>',
 '<action-date><date>20171207</date></action-date>',
 '<generating-office><country>US</country></generating-office>',
 '<classification-status>B</classification-status>',
 '<c

In [78]:
res_tup = retrieve_subtext_fast(6067926, 10, '../data/grants2012/ipg120417.xml')

In [79]:
res_tup[0]

10

In [80]:
res_tup[1][10]

'<doc-number>7945484</doc-number>'

In [83]:
list(index2017_we_need.keys())[0:5]

['15146848', '15192621', '15334971', '15049456', '15282523']

In [84]:
index2017_we_need['15146848']

[['application2017/ipa171207.xml', 5183637],
 ['application2017/ipa171207.xml', 5183673]]

In [91]:
res_tup = retrieve_subtext_fast(5183637, 5000, '../data/application2017/ipa171207.xml')

In [96]:
find_nearest_root_element(res_tup[0], res_tup[1], 'us-patent-application')

4667

In [93]:
res_tup[0]

5000

In [97]:
res_tup[1][4667]

'<us-patent-application lang="EN" dtd-version="v4.4 2014-04-03" file="US20170351915A1-20171207.XML" status="PRODUCTION" id="us-patent-application" country="US" date-produced="20171121" date-publ="20171207">'

In [103]:
res_pat = retrieve_begin_end(4667, res_tup[1], 'us-patent-application')

In [105]:
res_pat[0:5]

['<us-patent-application lang="EN" dtd-version="v4.4 2014-04-03" file="US20170351915A1-20171207.XML" status="PRODUCTION" id="us-patent-application" country="US" date-produced="20171121" date-publ="20171207">',
 '<us-bibliographic-data-application lang="EN" country="US">',
 '<publication-reference>',
 '<document-id>',
 '<country>US</country>']

In [104]:
res_pat[-1]

'</us-patent-application>'

In [ ]:
subtext = !sed -n ''

# Try and error

In [74]:
# create_one_index("grants2012/ipg120103.xml")

In [21]:
xmlrel = "application2017/ipa170105.xml"

In [22]:
xmlpath = "../data/"+xmlrel

In [29]:
indexfile=INDEX_PATH+xmlrel+".idx"

In [31]:
!mkdir -p {os.path.dirname(indexfile)}

In [32]:
!grep -n "doc-number" {xmlpath} | sed -e 's/\(.*\)<doc-number>\([^<]*\)<\/doc-number>.*/\1\2/' > {indexfile}

In [45]:
with open(indexfile, "r") as f:
    tupples = [l.rstrip("\n").split(":", 1) for l in f]

In [46]:
len(tupples)

31695

In [60]:
dic = {}

In [61]:
for tup in tupples:
    dic.setdefault(tup[1], []).append(int(tup[0]))

In [62]:
# some doc-number seems dup
dic["15265874"]

[3508, 3648]

In [ ]:
for tup in tupples:    
    if tup[1] not in dic:
        print(tup[1])
    dic[tup[1]] = 1    

### just read xml file is too slow. Give up using python

In [4]:
%%time

with open("../data/" + xmlpath, "r") as f:
    lnum = 0
    fpos = 0
    line = f.readline()
    while line:
        lnum += 1
        line = f.readline()
        fpos = f.tell()


KeyboardInterrupt: 

### Whoosh trial

Conclusion: indexing is too slow and we can't use it.

In [6]:
from whoosh.index import create_in
from whoosh.fields import *
import os, os.path


In [14]:
schema = Schema(path=ID(stored=True), line_num=NUMERIC(stored=True), pos=NUMERIC(stored=True), line=TEXT)

In [15]:
if not os.path.exists("indexdir"):
    os.mkdir("indexdir")

ix = index.create_in("indexdir", schema)

In [17]:
writer = ix.writer()

In [18]:
xmlpath = "application2017/ipa170105.xml"

In [21]:
with open("../data/" + xmlpath, "r") as f:
    lnum = 0
    fpos = 0
    line = f.readline()
    while line:
        lnum += 1
        writer.add_document(path=xmlpath, line=line, line_num=lnum, pos=fpos)
        line = f.readline()
        fpos = f.tell()

writer.commit()
ix.close()

KeyboardInterrupt: 